# Image Registration with SimpleITK and SimpleElastix

The segmentations were only done in one sequence of the examination, but should be transfered to all other sequences. 
However, there was movement between sequences, so segmentations do not really fit. 
Image registration with SimpleElastix can solve this issue

In [30]:
from pathlib import Path
import SimpleITK as sitk
import pandas as pd
import nrrd
import numpy as np
from tqdm import tqdm
import shutil

from trainlib.utils import load_config
from trainlib.viewer import BasicViewer, ListViewer

In [31]:
DATA_DIR = Path("/workspace/data/original").absolute()
df = pd.read_csv("/workspace/data/original/osteosarcoma.csv")

### Identify sequence labels have been created with
The spatial dimensions between the label and the sequence it has been created with should be the closest. 
First check, if labels are unique for each exam, or if multiple labels exist.

Read out shape of labels and images and store them in the df. 

In [32]:
def get_shape(fn):
    fn = str(fn)  # sitk does not support pathlib
    shape = nrrd.read_header(fn)["sizes"]
    return tuple(shape[-3:])


def get_distance(a, b):
    return sum([abs(_a - _b) for _a, _b in zip(a, b)])

In [33]:
df["image_shapes"] = [get_shape(DATA_DIR / fn) for fn in df.image]
df["label_shapes"] = [get_shape(DATA_DIR / fn) for fn in df.label]
df["distance"] = [get_distance(a, b) for a, b in zip(df.image_shapes, df.label_shapes)]

In [35]:
%%capture
patients = df.patient_id.unique()
exam_types = df.exam_type.unique()
sequences_list = []
for pat_id in patients:
    patient = df[df.patient_id == pat_id]
    for exam in exam_types:
        sequences = patient[patient.exam_type == exam]
        if len(sequences) == 0:
            continue
        assert len(sequences.label.unique()) == 1, sequences.label.unique()

        idx = np.where(sequences.distance == sequences.distance.min())[0][0]
        fixed_or_moving = ["moving"] * len(sequences)
        fixed_or_moving[int(idx)] = "fixed"
        sequences["fixed_or_moving"] = fixed_or_moving
        sequences_list += [sequences]

In [36]:
df = pd.concat(sequences_list)
df.to_csv("../data/original/osteosarcoma.csv", index=False)

### Example Registration 

Registration is performed in two steps: 

1. Resampling based solely on the DICOM header
2. Rigid transformation of the images

Idealy, the patient did not move between examinations
So resampling simply based on the header information should give the best results.
However, some patients were uneasy during the examinations and did move. 
Therefore, a rigid registration is appended to the initial resampling. 
Affine resampling is not performed, as it distorts the images to much, so that important image information is lost, and 
I believe this will affect performance of the deep learning algorithms. 


In [ ]:
sample = df[(df.patient_id == 50451996)]

In [ ]:
sample = sample[(sample.exam_type == "Baseline")]
fixed = sample.image[sample.fixed_or_moving == "fixed"]
moving = sample.image[sample.fixed_or_moving == "moving"]

In [39]:
results = []
fixed_image = sitk.ReadImage(str(DATA_DIR / fixed.item()))
parameterMap = sitk.GetDefaultParameterMap("rigid")

for m in tqdm(moving):
    moving_image = sitk.ReadImage(str(DATA_DIR / m))
    moving_image_resampled = sitk.Resample(
        moving_image, fixed_image
    )  # first simple Resampling with DICOM header information
    results_image = sitk.Elastix(fixed_image, moving_image_resampled, parameterMap)
    results += [moving_image_resampled, fixed_image, results_image]

100%|█████████████████████████████████████████████| 4/4 [00:14<00:00,  3.71s/it]


In [40]:
segmentation, _ = nrrd.read(str(DATA_DIR / list(sample.label)[0]))
segmentation[0] *= 2
segmentation = segmentation.max(0)

In [41]:
ListViewer(
    [sitk.GetArrayFromImage(im) for im in results],
    [segmentation.transpose(2, 1, 0)] * len(results),
    figsize=(3, 3),
).show()

## Duplicate data

Original data is immutable. As some files need to be changed, first duplicates are created. From now on, only the duplicated data is used. 

In [42]:
DATA_DIR2 = DATA_DIR.parent / "processed"
if not DATA_DIR2.exists():
    shutil.copytree(DATA_DIR, DATA_DIR2)

### Correct segmentations for images and labels with unequal shape
The shape of the fixed image and the label should be identical. 
However, this is not always the case. Reasons for this might include:

    (A) Annotations performed with mutliple sequences in 3D Slicer.  
    (B) Cropping by the `seg.nrrd` format.  
    (C) Human errors.   
    
Spatial information should be perserved in the `seg.nrrd` file, so Resampling should work for these cases. 
As all sequences will be resampled to the fixed image, it also makes the most sense to resample the label to the fixed image and not vice versa. 


### Run Registration for all images

In [48]:
DATA_DIR3 = DATA_DIR2.parent / "resampled"
DATA_DIR3.mkdir(exist_ok=True)

In [49]:
parameterMap = sitk.GetDefaultParameterMap("rigid")

for pat_id in tqdm(patients):
    patient = df[df.patient_id == pat_id]
    for exam in exam_types:
        sequences = patient[patient.exam_type == exam]
        if len(sequences) == 0:
            continue
        fixed = sequences.image[sequences.fixed_or_moving == "fixed"].item()
        moving = sequences.image[sequences.fixed_or_moving == "moving"]
        segmentation = Path(list(sequences.label)[0])

        # resample and register images
        fixed_image = sitk.ReadImage(str(DATA_DIR2 / fixed))
        if not (DATA_DIR3 / fixed).parent.exists():
            (DATA_DIR3 / fixed).parent.mkdir(parents=True)
        sitk.WriteImage(fixed_image, str(DATA_DIR3 / fixed))

        for m in moving:
            if not (DATA_DIR3 / m).parent.exists():
                (DATA_DIR3 / m).parent.mkdir(parents=True)

            if (DATA_DIR3 / m).exists():
                continue

            moving_image = sitk.ReadImage(str(DATA_DIR2 / m))
            # First simple Resampling with DICOM header information
            moving_image_resampled = sitk.Resample(moving_image, fixed_image)
            # Now use SimpleElastix for final adjustment
            results_image = sitk.Elastix(fixed_image, moving_image_resampled, "rigid")
            assert results_image.GetSize() == fixed_image.GetSize(), "Size mismatch"
            sitk.WriteImage(results_image, str(DATA_DIR3 / m))

100%|███████████████████████████████████████████| 50/50 [19:08<00:00, 22.97s/it]


In a next step the images are resampled to isotrophic voxel sizes of 1 x 1 x 1 mm and a uniform orientation. 

## Convert Segmentations

The segmentations have two channels, one containing bone and tumor, one just tumor. These will be combined to make handling the files easier. Also the affine matrix has to be adapted. The affine matrix in the seg.nrrd files has dimension (4,3), because the file has two channels. 
However, the first row is `nan`. This leads to problems when reading the file with monais `NrrdReader`, during LPS to RAS conversion. 
The easiest solution is to remove the first row in the affine matrix, making it shape (3,3). 

In [50]:
for fn in tqdm(df.label.unique()):
    label, header = nrrd.read(str(DATA_DIR2 / fn))
    if label.shape[0] == 2:
        label[0] *= 2
        label = label.max(0)
    if header["space directions"].shape == (4, 3):
        header["sizes"] = header["sizes"][1:]
        header["kinds"] = header["kinds"][1:]
        header["space directions"] = header["space directions"][1:]
        header["dimension"] = 3
    nrrd.write(str(DATA_DIR3 / fn), label, header)

100%|█████████████████████████████████████████| 114/114 [01:50<00:00,  1.03it/s]


In [51]:
df.to_csv(DATA_DIR3 / "osteosarcoma.csv", index=False)